In [1]:
import owlready2 as owl2
from urllib import request
import os

## Genereal Information


### Problem 1:

The IRI of the time ontology  <http://www.w3.org/2006/time> refers to a file in turtle syntax. This syntax is currently (2021-01-22) not supported by owlready. Thus, this ontology canont be loaded (directly) with owlready.

### Problem 2:

Another ontologies ABC which are in the supported "RDF_XML"-format might import that ontology (via `<owl:imports rdf:resource="http://www.w3.org/2006/time"/>`). Owlready detects the import, tries to resolve it and fails due to problem 1.


## Solution (Problem 1):

Show how to convert that file programmatically (i.e. without using a GUI like protégé).

- assumes https://github.com/sszuev/ont-converter to be installed (requires local build with *maven*)
- `ont-converter.jar` must be located in current dir

In [2]:
# download the original file

url = "http://www.w3.org/2006/time"
res = request.urlopen(url)
t_path = "time_turtle.txt"
with open(t_path, "wb") as t_file:
    # write plain bytes (without de- and encoding)
    t_file.write(res.read())

In [3]:
# convert from Turtle to RDF_XML

o_path = "time.owl"
cmd = f"java -jar ont-converter.jar  -i {t_path} -if TURTLE -o {o_path} -of RDF_XML"
os.system(cmd)

0

In [4]:
fpath = "time.owl"
onto = owl2.get_ontology(fpath).load()

* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#inXSDDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#xsdDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...


In [5]:
list(onto.classes())

[time.DayOfWeek,
 time.Duration,
 time.TemporalDuration,
 time.DateTimeInterval,
 time.ProperInterval,
 time.TimeZone,
 time.Instant,
 time.Interval,
 time.MonthOfYear,
 time.TemporalUnit,
 time.DateTimeDescription,
 time.TemporalEntity,
 time.GeneralDateTimeDescription,
 time.DurationDescription,
 time.GeneralDurationDescription,
 time.TemporalPosition,
 time.TRS,
 time.TimePosition,
 time.Year,
 time.January]

## Solution (Problem 2):

In [6]:
world2 = owl2.World()

fpath2 = "turtle_import.owl"

try: 
    world2.get_ontology(fpath2).load()
except owl2.OwlReadyOntologyParsingError:
    print(f"could not import '{fpath2}'" )

could not import 'turtle_import.owl'


In [7]:
world3 = owl2.World()

# load the RDF_XML-version of the time ontology first
time_onto = world2.get_ontology(fpath).load()

# load the ontology which imports the time ontology second
test_onto = world2.get_ontology(fpath2).load()

* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#inXSDDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://www.w3.org/2006/time#xsdDateTime belongs to more than one entity types: [owl.DeprecatedProperty, owl.DatatypeProperty]; I'm trying to fix it...


In [8]:
# manualy add the imported ontology to the respective data-structure

test_onto.imported_ontologies.append(time_onto)

In [9]:
from io import StringIO, BytesIO

In [10]:
## o1.imported_ontologies = [o2]
file = BytesIO()
test_onto.save(file)
print(file.getvalue().decode("utf8"))

<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:owl="http://www.w3.org/2002/07/owl#"
         xml:base="http://www.abc.net/turtle_import"
         xmlns="http://www.abc.net/turtle_import#">

<owl:Ontology rdf:about="http://www.abc.net/turtle_import">
  <owl:imports rdf:resource="http://www.w3.org/2006/time"/>
</owl:Ontology>

<owl:Class rdf:about="#thingB"/>

<owl:Class rdf:about="#C">
  <rdfs:subClassOf rdf:resource="http://www.w3.org/2006/time#Year"/>
</owl:Class>


</rdf:RDF>



In [11]:
# confirm that the Class C is indeed a subclass of the 

test_onto.C.is_a

[time.Year]

In [12]:
test_onto.C.is_a == [time_onto.Year]

True